In [12]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from fake_useragent import UserAgent

user_agent = UserAgent()
main_url = "https://stackoverflow.com/questions/tagged/python"


page_limit = 5
page_number = 1


scraped_data = []
def question_scrape(next_url):
    page = requests.get(next_url,user_agent.chrome)
    soup = BeautifulSoup(page.text,'lxml')
    all_divs = soup.find_all('div',class_='s-post-summary')
    for div in all_divs:
        question_link = f"https://stackoverflow.com/"+ div.find('a').get('href')+"/"
        question = div.find('h3',{'class':'s-post-summary--content-title'}).text.strip('\n')
        votes_count = div.find('div',{'class':'s-post-summary--stats-item s-post-summary--stats-item__emphasized'}).text.split('\n')[1] + " votes"
        answers_count = div.find_all('span',{'class':'s-post-summary--stats-item-number'})[1].text + " answers"
        views_count = div.find_all('span',{'class':'s-post-summary--stats-item-number'})[2].text + " views"
        user_name = div.find('div',{'class':'s-user-card--link'}).contents[1].text
        
        data = [question_link,question,votes_count,answers_count,views_count,user_name]
        scraped_data.append(data)    

    
    scraped_data_df = pd.DataFrame(scraped_data,columns=['Question Link','Question','Votes Count','Answers Count','Views Count','Username'])
    scraped_data_df.index+=1

    scraped_data_df.to_csv('Stack_Questions.csv',index=True)


while page_number <= page_limit :
    url = main_url + "?tab=newest&page=" + str(page_number)
    question_scrape(url)
    page_number+=1
